In [1]:
import os

os.chdir('../')

#import sys
#sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn

from autoneuro.core import build_from_yaml
from autoneuro import domains

In [2]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
output_notebook()

Loading BokehJS ...

In [3]:
all_neuro = pd.read_excel('data/neuro_data.xlsx',
                          na_values=['na', '-', 'nd']).groupby(['VisitDate', 'PatientID']).first()
#all_neuro.set_index(['VisitDate', 'PatientID'], inplace=True)
all_neuro.head()

Visit  Age Sex  Race Race_specify  Education  \
VisitDate  PatientID                                                 
2013-09-24 A0091        1.0   36   1     2          NaN         16   
           A0107        1.0   49   2     2          NaN         12   
           A0138        1.0   53   2     2          NaN         11   
2013-10-03 A0165        1.0   62   1     2          NaN         12   
2013-10-08 A0127        1.0   62   1     2          NaN         16   

                      EngFluency  SpanFluency  MMSE  DigitBackwardSENAS  ...  \
VisitDate  PatientID                                                     ...   
2013-09-24 A0091             3.0          0.0  27.0                 6.0  ...   
           A0107             3.0          0.0  27.0                 3.0  ...   
           A0138             3.0          0.0   NaN                 NaN  ...   
2013-10-03 A0165             3.0          0.0  29.0                 5.0  ...   
2013-10-08 A0127             3.0          0.0  28.0                 2.0  ...   

                      Professional  Homeowner  Householdsize  Earnings.Indiv  \
VisitDate  PatientID                                                           
2013-09-24 A0091               NaN        NaN            NaN             NaN   
           A0107               NaN        NaN            NaN             NaN   
           A0138               NaN        NaN            NaN             NaN   
2013-10-03 A0165               NaN        NaN            NaN             NaN   
2013-10-08 A0127               NaN        NaN            NaN             NaN   

                      Income.Indiv  Earnings.House  Income.House  \
VisitDate  PatientID                                               
2013-09-24 A0091               NaN             NaN           NaN   
           A0107               NaN             NaN           NaN   
           A0138               NaN             NaN           NaN   
2013-10-03 A0165               NaN             NaN           NaN   
2013-10-08 A0127               NaN             NaN           NaN   

                      Income.House.Weighted1  Income.House.Weighted2  Effort  
VisitDate  PatientID                                                          
2013-09-24 A0091                         NaN                     NaN     1.0  
           A0107                         NaN                     NaN     NaN  
           A0138                         NaN                     NaN     1.0  
2013-10-03 A0165                         NaN                     NaN     NaN  
2013-10-08 A0127                         NaN                     NaN     NaN  

[5 rows x 163 columns]

In [4]:
COLUMN_MAPPING = {'HVLT.total': 'HVLTtotal',
                  'HVLT.recog': 'HVLTrecog',
                  'HVLT.delay': 'HVLTdelay'}

domain = domains.MemoryDomain.from_defaults(root_data = 'data/')
domain.load_data(all_neuro, mapping=COLUMN_MAPPING)
domain.check_reasonableness(drop=True)
domain.norm.dropna().head()

EXTREME OUTLIERS FOUND!
Dropping


,,BVMTimmed,BVMTdelay,BVMTrecog,HVLTtotal,HVLTdelay,HVLTrecog
VisitDate,PatientID,,,,,,
2019-11-05,B0709,-1.749474,-2.341317,-6.785714,-2.040632,-1.921569,-0.763780
2019-11-12,B0059,-0.940154,-0.520202,-1.886364,-2.749367,-5.815029,-1.111801
2019-11-19,B0711,-0.967078,-0.394286,0.400000,-2.276543,-2.480769,-3.915094
2019-11-20,B0712,-0.167954,0.489899,-1.886364,-1.483544,-0.612717,-1.111801
2019-11-26,B0417,-1.712355,-2.540404,-4.159091,-1.989873,-4.658960,-0.490683


In [5]:
domain.norm.describe()

,BVMTimmed,BVMTdelay,BVMTrecog,HVLTtotal,HVLTdelay,HVLTrecog
count,210.000000,256.000000,254.000000,39.000000,39.000000,13.000000
mean,-1.772083,-0.579738,-1.385338,-5.073968,-4.171895,-1.187592
std,1.362616,2.818632,2.228034,2.151731,1.760408,1.292657
min,-5.400881,-5.493421,-6.785714,-7.053165,-5.815029,-3.915094
25%,-2.702652,-2.680000,-2.889398,-6.800000,-5.236994,-1.732919
50%,-1.752578,-1.130742,0.166667,-6.103837,-4.884615,-1.084906
75%,-0.883629,0.580488,0.393294,-2.759869,-3.403604,-0.763780
max,1.547310,7.560606,0.500000,0.035443,-0.034682,0.751553


In [6]:
FIG_SIZE = {'plot_height': 300,
            'plot_width': 300}

delay_fig, delay_immed_fig, immed_recog_fig = domain.build_figs(fig_kwargs=FIG_SIZE)


new_fig = domain.build_scatter_fig(x = 'HVLTtotal',
                                   y = 'HVLTdelay',
                                   fig={'title': 'HVLT Total v delay',
                                        **FIG_SIZE})


show(layout([[delay_immed_fig, delay_fig],
             [immed_recog_fig, new_fig]]))



In [ ]:
new_fig = domain.build_scatter_fig(x = 'HVLTtotal',
                                   y = 'HVLTdelay',
                                   fig={'title': 'HVLT Total v delay',
                                        **FIG_SIZE})